# Community detection in Graphs

We will explore the different community detection algorithms in graphs. 

Not all fo them fit with our problem requirements, we will focus on the algorithms which fit the following requirements
* Directed edges.
* Homogeneous nodes. 
* Possible Weighted relationships (in the future).

The algorithms which fit the requirements are:
* Conductance metric
* K-1 Coloring
* Label Propagation
* Louvain
* Modularity metric
* Modularity Optimization
* Strongly Connected Components
* Weakly Connected Components

We will use an additional approach after embedding the graph, we have use Node2Vector and DeepWalk
* K-Means
* DBSCAN

We will explore each one of them, how they work, their advantages and disadvantages and when to use them. 

Check this documentation to understand and explore the Community Detection algorithms:
https://neo4j.com/docs/graph-data-science/current/algorithms/community/

### Conductance metric 

### K-Means Clustering for Community Detection 
Read the embedding graph and apply clustering with cuML (RAPIDS) to use GPU

In [1]:
import torch
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Read embedding representation
node2vector_embeddings_filepath = 'deepwalk_embeddings.pt'
# Load the tensor
node2vector_embeddings = torch.load(node2vector_embeddings_filepath)
# Ensure the tensor is moved to CPU and converted to a NumPy array
node2vector_embeddings = node2vector_embeddings.cpu().numpy()
node2vector_embeddings

C:\Users\PC\AppData\Local\Temp\ipykernel_44556\3925009371.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  node2vector_embeddings = torch.load(node2vector_embeddings_file

array([[ 0.140931  ,  0.13705331,  0.08320353, ...,  0.11453004,
        -0.01445364, -0.07224333],
       [-0.16662396, -0.17447558, -0.00990902, ..., -0.17300689,
         0.08206606, -0.11753833],
       [-0.02905227, -0.11975029, -0.0781097 , ..., -0.26019233,
         0.09917527,  0.0829257 ],
       ...,
       [ 0.01846431,  0.11540488, -0.0927221 , ..., -0.07000463,
         0.07836554, -0.17765656],
       [-0.09754419,  0.21097174, -0.16383187, ..., -0.02562554,
        -0.01701593,  0.03703906],
       [-0.15740357,  0.05840908, -0.16658461, ...,  0.16704065,
         0.10804104,  0.17264745]], dtype=float32)

#### Normalize data to use K-means
k-means is sensitive to features in different scales.
We are using MinMax scale to avoid distortion in the distributions generated by the embedding process (Standard Scale or other could distort them)

NOTE: Check what happen when Scale is not applied.  

In [8]:
# normalize embedding vectors
scaler = MinMaxScaler()
scaler.fit(node2vector_embeddings)
node2vector_embeddings = scaler.transform(node2vector_embeddings)
node2vector_embeddings

array([[0.5984971 , 0.5310766 , 0.47160393, ..., 0.517416  , 0.51933837,
        0.4729978 ],
       [0.42754388, 0.42757073, 0.42093855, ..., 0.37706408, 0.57293135,
        0.45051533],
       [0.5040126 , 0.44575328, 0.3838285 , ..., 0.33450726, 0.58243126,
        0.55001706],
       ...,
       [0.5304245 , 0.5238839 , 0.37587744, ..., 0.42734134, 0.5708766 ,
        0.42067525],
       [0.4659416 , 0.5556361 , 0.33718443, ..., 0.44900358, 0.51791567,
        0.5272409 ],
       [0.43266898, 0.50494695, 0.33568656, ..., 0.54304737, 0.587354  ,
        0.594551  ]], dtype=float32)

In [4]:
# calculate the inertia for different clusters
inertia = []
max_n_clusters = 5800
K = range(1, max_n_clusters)

# Fit KMeans and calculate inertia for each k
for k in K:
    print(f'n clusters: {k}')
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(node2vector_embeddings)
    inertia.append(kmeans.inertia_)

n clusters: 1
n clusters: 2
n clusters: 3
n clusters: 4
n clusters: 5
n clusters: 6
n clusters: 7
n clusters: 8
n clusters: 9
n clusters: 10
n clusters: 11
n clusters: 12
n clusters: 13
n clusters: 14
n clusters: 15
n clusters: 16
n clusters: 17
n clusters: 18
n clusters: 19
n clusters: 20
n clusters: 21
n clusters: 22
n clusters: 23
n clusters: 24
n clusters: 25
n clusters: 26
n clusters: 27
n clusters: 28
n clusters: 29
n clusters: 30
n clusters: 31
n clusters: 32
n clusters: 33
n clusters: 34
n clusters: 35
n clusters: 36
n clusters: 37
n clusters: 38
n clusters: 39
n clusters: 40
n clusters: 41
n clusters: 42
n clusters: 43
n clusters: 44
n clusters: 45
n clusters: 46
n clusters: 47
n clusters: 48
n clusters: 49
n clusters: 50
n clusters: 51
n clusters: 52
n clusters: 53
n clusters: 54
n clusters: 55
n clusters: 56
n clusters: 57
n clusters: 58
n clusters: 59
n clusters: 60
n clusters: 61
n clusters: 62
n clusters: 63
n clusters: 64
n clusters: 65
n clusters: 66
n clusters: 67
n cl

In [7]:
# Plot the inertia values vs # clusters
plt.figure(figsize=(10, 6))
plt.plot(K, inertia, 'bx-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('The Elbow plot. Inertia vs # Clusters')
plt.show()

Once the Elbow is plot, we can define the best number of clusters to find the communities

Based on the elbow plot (applying normalization), there isn't a clear value to select there isn't a clear elbow. 
There are several reasons which support that:   
* The nodes embedding dimensions are not representing properly the data patterns, they may be so close or so far between nodes. 
* The nodes embedding representation is heterogeneous, there isn't a clear pattern to get maximum cohesion and maximum separation. 
* K-Means is no the best algorithm to find the communities.
* Normalization may not be needed, WARNING we need to define why and over what conditions. Analyze the trade-off

In [9]:
# Define the communities
n_clusters = 3000
kmeans_model = KMeans(n_clusters=n_clusters, random_state=42)
kmeans_model.fit(node2vector_embeddings)

labels = kmeans_model.labels_
print(labels)

[2195 2875  875 ... 1429 1892 2933]


In [31]:
list(labels)

[np.int32(51),
 np.int32(17),
 np.int32(54),
 np.int32(20),
 np.int32(13),
 np.int32(40),
 np.int32(32),
 np.int32(26),
 np.int32(27),
 np.int32(25),
 np.int32(64),
 np.int32(33),
 np.int32(69),
 np.int32(35),
 np.int32(59),
 np.int32(4),
 np.int32(5),
 np.int32(22),
 np.int32(8),
 np.int32(69),
 np.int32(44),
 np.int32(18),
 np.int32(7),
 np.int32(53),
 np.int32(36),
 np.int32(11),
 np.int32(38),
 np.int32(26),
 np.int32(28),
 np.int32(20),
 np.int32(38),
 np.int32(4),
 np.int32(23),
 np.int32(26),
 np.int32(59),
 np.int32(54),
 np.int32(22),
 np.int32(39),
 np.int32(5),
 np.int32(54),
 np.int32(42),
 np.int32(40),
 np.int32(3),
 np.int32(5),
 np.int32(48),
 np.int32(42),
 np.int32(20),
 np.int32(10),
 np.int32(57),
 np.int32(55),
 np.int32(64),
 np.int32(36),
 np.int32(25),
 np.int32(13),
 np.int32(46),
 np.int32(4),
 np.int32(11),
 np.int32(8),
 np.int32(57),
 np.int32(46),
 np.int32(11),
 np.int32(13),
 np.int32(59),
 np.int32(49),
 np.int32(48),
 np.int32(32),
 np.int32(4),
 np.in

In [7]:
len(labels)

19129

In [ ]:
servers_data_filepath = './data/servers_data.csv'
servers_df = pd.read_csv(servers_data_filepath)
servers_df